In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/'My Drive'/nba_pred/
%ls

/content/drive/My Drive/nba_pred
2019_2020/            away_scores    home_data            model0.ipynb
append_dataset.ipynb  Curr/          home_scores          model1.ipynb
away_data             data_info.txt  init_data_set.ipynb


In [12]:
def prediction(home, away, month, season):
  stats = getCurr(home, away, month, season)
  data1, data2, score1, score2 = getData(True)
  REG_ACC = []
  NN_ACC = []
  reg_pred = []
  nn_pred = []

  print('With the '+ home + ' being the home team')
  print('Running model 7 times...')
  print('loading', end="")
  for i in range(7):
    reg, model, acc1, acc2 = getModel(data1, data2, score1, score2)
    REG_ACC.append(acc1)
    NN_ACC.append(acc2)
    diff1, diff2 = getPredict(reg, model, stats)
    reg_pred.append(diff1)
    nn_pred.append(diff2)
    print('.', end="")
  print()
  print('out of 7 runs')
  print('Ridge prediction {:.2f}% predicted {} wins for the '.format(avg(REG_ACC) * 100, countW(reg_pred)) + home + ', by avg of {:.2f} pts'.format(avg(reg_pred)))
  print('NN prediction {:.2f}% predicted {} wins for the '.format(avg(NN_ACC) * 100, countW(nn_pred)) + home + ', by avg of {:.2f} pts'.format(avg(nn_pred)))

  #switch home and away
  print()
  stats = getCurr(home, away, month, season)
  data1, data2, score1, score2 = getData(False)
  REG_ACC = []
  NN_ACC = []
  reg_pred = []
  nn_pred = []

  
  print('With the '+ away + ' being the home team')
  print('Running model 7 times...')
  print('loading', end="")
  for i in range(7):
    reg, model, acc1, acc2 = getModel(data1, data2, score1, score2)
    REG_ACC.append(acc1)
    NN_ACC.append(acc2)
    diff1, diff2 = getPredict(reg, model, stats)
    reg_pred.append(diff1)
    nn_pred.append(diff2)
    print('.', end="")
  print()
  print('out of 7 runs')
  print('Ridge prediction {:.2f}% predicted {} wins for the '.format(avg(REG_ACC) * 100, countW(reg_pred)) + away + ', by avg of {:.2f} pts'.format(avg(reg_pred)))
  print('NN prediction {:.2f}% predicted {} wins for the '.format(avg(NN_ACC) * 100, countW(nn_pred)) + away + ', by avg of {:.2f} pts'.format(avg(nn_pred)))

def getCurr(home, away, month, season):
  import pandas as pd
  import numpy as np

  team_data_trad = pd.read_excel('{}/{}/TeamStatsTrad_{}.xlsx'.format(season, month, month)).dropna()
  team_data_advan = pd.read_excel('{}/{}/TeamStatsAdvan_{}.xlsx'.format(season, month, month)).dropna()
  team_data_misc = pd.read_excel('{}/{}/TeamStatsMisc_{}.xlsx'.format(season, month, month)).dropna()
  team_data_4 = pd.read_excel('{}/{}/TeamStats4_{}.xlsx'.format(season, month, month)).dropna()
  team_data_score = pd.read_excel('{}/{}/TeamStatsScore_{}.xlsx'.format(season, month, month)).dropna()
  team_data_opp = pd.read_excel('{}/{}/TeamStatsOpp_{}.xlsx'.format(season, month, month)).dropna()

  team_name = list(team_data_trad.columns)
  team_name.pop(0)

  team_stat = {}
  for name in team_name:
    team_stat[name] = list(team_data_trad[name])
    team_stat[name]+=(list(team_data_advan[name]))
    team_stat[name]+=(list(team_data_misc[name]))
    team_stat[name]+=(list(team_data_4[name]))
    team_stat[name]+=(list(team_data_score[name]))
    team_stat[name]+=(list(team_data_opp[name]))
  
  stats1 = [*team_stat[home], *team_stat[away]]
  stats2 = [*team_stat[away], *team_stat[home]]

  matrix = []
  matrix.append(stats1)
  matrix.append(stats2)

  matrix = np.matrix(matrix)
  #print(matrix.shape) #VERIFICATION

  return matrix

def avg(diff):
  return float(sum(diff)/len(diff))

def countW(diff):
  win = 0
  for i in diff:
    if i > 0:
      win = win + 1
  
  return win

def getData(show):
  import pandas as pd
  import numpy as np
  data1 = np.loadtxt('home_data', delimiter=',')
  data2 = np.loadtxt('away_data', delimiter=',')
  score1 = np.loadtxt('home_scores', delimiter=',')
  score2 = np.loadtxt('away_scores', delimiter=',')

  score1 = np.reshape(score1, (len(score1), 1))
  score2 = np.reshape(score2, (len(score2), 1))
  
  if show:
    print("Current Database size: ", end="")
    print(data1.shape[0]) #VERIFICATION
  #print(score1.shape) #VERIFICATION

  return data1, data2, score1, score2

def getModel(data1, data2, score1, score2):
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import normalize
  import tensorflow as tf
  from tensorflow.keras.layers import Dense
  from sklearn.metrics import accuracy_score
  from sklearn.linear_model import Ridge
  import pandas as pd
  import numpy as np

  X_train1, X_test_home, y_train1, y_test_home = train_test_split(data1, score1, test_size=0.25, shuffle=True)
  X_train2, X_test_away, y_train2, y_test_away = train_test_split(data2, score2, test_size=0.25, shuffle=True)
  train_matrix = np.concatenate((X_train1, X_train2), axis=0)
  train_target = np.concatenate((y_train1, y_train2), axis=None)

  win_loss = []
  for home, away in zip(y_test_home, y_test_away):
    if (home > away):
      win_loss.append(1)
    else:
      win_loss.append(0)
  
  #ridge model
  regression = Ridge()
  regression.fit(train_matrix, train_target)
  test_home = regression.predict(X_test_home)
  test_away = regression.predict(X_test_away)

  pred_win_loss = []
  for home, away in zip(test_home, test_away):
    if (home > away):
      pred_win_loss.append(1)
    else:
      pred_win_loss.append(0)

  acc1 = accuracy_score(win_loss, pred_win_loss)
  #print('acc REG: ' + str(accuracy_score(win_loss, pred_win_loss)))

  #NN model
  model = tf.keras.Sequential()
  model.add(Dense(168, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='softplus'))
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

  history = model.fit(train_matrix, train_target, verbose=0, batch_size=1, epochs=3)

  test_home = model.predict(X_test_home)
  test_away = model.predict(X_test_away)

  pred_win_loss = []
  for home, away in zip(test_home, test_away):
    if (home > away):
      pred_win_loss.append(1)
    else:
      pred_win_loss.append(0)

  acc2 = accuracy_score(win_loss, pred_win_loss)
  #print('acc NN: ' + str(accuracy_score(win_loss, pred_win_loss)))
  return regression, model, acc1, acc2

def getPredict(reg, model, stats):
  reg_pred = reg.predict(stats)
  mod_pred = model.predict(stats)

  diff1 = float(reg_pred[0] - reg_pred[1])
  diff2 = float(mod_pred[0] - mod_pred[1])

  return diff1, diff2

In [13]:
if __name__ == '__main__':
  #which data to use
  season = '2019_2020'
  month = 'Jul20'

  prediction('Portland Trail Blazers', 'Boston Celtics', month, season)
  

Current Database size: 971
With the Portland Trail Blazers being the home team
Running model 7 times...
loading.......
out of 7 runs
Ridge prediction 67.90% predicted 3 wins for the Portland Trail Blazers, by avg of 0.29 pts
NN prediction 68.14% predicted 7 wins for the Portland Trail Blazers, by avg of 7.37 pts

With the Boston Celtics being the home team
Running model 7 times...
loading.......
out of 7 runs
Ridge prediction 66.37% predicted 6 wins for the Boston Celtics, by avg of 9.02 pts
NN prediction 67.61% predicted 7 wins for the Boston Celtics, by avg of 6.08 pts
